In [2]:
import os

api_key = os.environ["OPENAI_API_KEY"]

In [3]:
import pandas as pd
import tiktoken
import os
import openai
import pickle
from langchain.embeddings import OpenAIEmbeddings

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader

In [4]:
with open('restaurent_docs_1.pickle', 'rb') as f:
    business_df = pickle.load(f)

In [5]:
business_df.shape

(4214, 17)

In [6]:
business_df = business_df.dropna(subset=['description'])

In [7]:
business_df.shape

(4214, 17)

In [8]:
business_df.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,text,doc_information
0,Lucky Thai Cuisine,"Lucky Thai Cuisine, 7239 Fairbanks North Houst...",0x8640cffdfa4b6e11:0x76fb95fddc6392cd,Down-to-earth restaurant featuring traditional...,29.879276,-95.525232,[Thai restaurant],4.8,37,0,"[[Tuesday, 11AM–8PM], [Wednesday, 11AM–8PM], [...","{'Service options': ['Takeout', 'Dine-in', 'De...",Closed ⋅ Opens 11AM,None,https://www.google.com/maps/place//data=!4m2!3...,"Found this place by accident, decided to try i...",Lucky Thai Cuisine Down-to-earth restaurant fe...
1,sweetgreen,"sweetgreen, 600 N Shepherd Dr Suite 149, Houst...",0x8640c7c278e09d8f:0xc5f6d7b9939e6b69,Locavore-friendly counter-serve chain speciali...,29.782347,-95.408211,"[Health food restaurant, Delivery service, Tak...",4.3,28,2,"[[Saturday, 10:30AM–9PM], [Sunday, 10:30AM–9PM...","{'Service options': ['No-contact delivery', 'D...",Open ⋅ Closes 9PM,None,https://www.google.com/maps/place//data=!4m2!3...,Accidentally arrived a half hour early for my ...,sweetgreen Locavore-friendly counter-serve cha...
2,Degust,"Degust, 7202 Long Point Rd, Houston, TX 77055",0x8640c7c1d7edfedf:0x87efc5a752804495,Cozy farm-to-table restaurant with a show kitc...,29.803281,-95.474054,[Small plates restaurant],4.9,28,0,"[[Monday, Closed], [Tuesday, Closed], [Wednesd...","{'Service options': ['Dine-in', 'Delivery', 'T...",Closed ⋅ Opens 5:30PM Thu,None,https://www.google.com/maps/place//data=!4m2!3...,Passionate staff delivers an exceptional tasti...,Degust Cozy farm-to-table restaurant with a sh...
3,Hunan's Restaurant,"Hunan's Restaurant, 3835 Bellaire Blvd, Housto...",0x8640c111869fd267:0xc32f15afbb4cc24c,Chinese classics such as chow mein & chop suey...,29.705374,-95.437983,"[Restaurant, Chinese restaurant, Hunan restaur...",4.0,228,0,"[[Friday, 11AM–10PM], [Saturday, 11:30AM–10PM]...","{'Service options': ['No-contact delivery', 'D...",Open ⋅ Closes 10PM,"[0x8640bff9aa9d31c9:0x5a55e94b6eebd8ae, 0x8640...",https://www.google.com/maps/place//data=!4m2!3...,My baby was craving some Chinese food and Mong...,Hunan's Restaurant Chinese classics such as ch...
4,Speedy Stop,"Speedy Stop, 4981 Hwy 6 N, Houston, TX 77084",0x8640d9ff4fdee403:0x8a2ee7569adfaf82,"Convenience-store chain with drinks & food, in...",29.848533,-95.646671,"[Convenience store, Car wash, Fast food restau...",2.8,18,0,"[[Friday, Open 24 hours], [Saturday, Open 24 h...","{'Service options': ['Delivery', 'In-store pic...",Open 24 hours,"[0x8640d9ff4fdee403:0xf7dd9c8f35899fc4, 0x8640...",https://www.google.com/maps/place//data=!4m2!3...,"Love this location - bright, clean, friendly s...",Speedy Stop Convenience-store chain with drink...


In [9]:
columns_to_drop = ['relative_results', 'doc_information', 'url', 'latitude', 'longitude']

business_df.drop(columns=columns_to_drop, inplace=True)

In [10]:
docs_df = business_df

In [11]:
def concat_dict_values(d):
    if d is None:
        return ''
    return ', '.join([f"{' '.join(value)}" for key, value in d.items()])


docs_df['combined_info'] = 'Name: ' +docs_df['name']+' Description: '+ docs_df['description'].fillna(' ') + ' Category: ' + docs_df['category'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x)) + ' Service Options: ' + docs_df['MISC'].apply(concat_dict_values)  + ' Combined Reviews: ' + + docs_df['text'].fillna('')

In [12]:
docs_df.isna().any()

name              False
address           False
gmap_id           False
description       False
category          False
avg_rating        False
num_of_reviews    False
price             False
hours             False
MISC              False
state             False
text              False
combined_info     False
dtype: bool

In [13]:
docs_df = docs_df.astype(str)


In [14]:
from langchain_community.document_loaders import DataFrameLoader
loader = DataFrameLoader(docs_df, page_content_column="combined_info")
data = loader.load()

In [15]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

Created a chunk of size 2671, which is longer than the specified 1000
Created a chunk of size 1743, which is longer than the specified 1000
Created a chunk of size 1195, which is longer than the specified 1000
Created a chunk of size 9678, which is longer than the specified 1000
Created a chunk of size 2416, which is longer than the specified 1000
Created a chunk of size 14566, which is longer than the specified 1000
Created a chunk of size 1327, which is longer than the specified 1000
Created a chunk of size 4486, which is longer than the specified 1000
Created a chunk of size 15786, which is longer than the specified 1000
Created a chunk of size 1087, which is longer than the specified 1000
Created a chunk of size 1147, which is longer than the specified 1000
Created a chunk of size 2444, which is longer than the specified 1000
Created a chunk of size 1459, which is longer than the specified 1000
Created a chunk of size 1481, which is longer than the specified 1000
Created a chunk of

In [16]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
llm = OpenAI(openai_api_key=api_key)

#Vector DB
docsearch = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db_res2')

/Users/nachiket/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/nachiket/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [17]:
query = "I'm looking for indian food. What could you suggest to me?"
docs = docsearch.similarity_search(query, k=3)
docs

[Document(page_content="But I would be doing you a disservice if I didn't tell you that this is some of the most delicious Indian food I have ever had. I'm always trying different things from their menu it hits the spot and I normally order from them on doordash or take out.", metadata={'MISC': "{'Service options': ['Outdoor seating', 'Delivery', 'Takeout'], 'Health & safety': ['Reservations required'], 'Highlights': ['Fireplace', 'Live music'], 'Offerings': ['Alcohol', 'Coffee', 'Comfort food', 'Organic dishes', 'Salad bar'], 'Dining options': ['Breakfast'], 'Amenities': ['Good for kids', 'High chairs'], 'Payments': ['Cash-only']}", 'address': 'Hyderabad House, 6439 Westheimer Rd, Houston, TX 77057', 'avg_rating': '3.8', 'category': "['Indian restaurant', 'Bakery', 'Caterer', 'Restaurant']", 'description': 'Local Hyderabadi-Indian chain offering biryani & other classic dishes, plus lunch & dinner buffets.', 'gmap_id': '0x8640c3c768184389:0x318b142255a78eda', 'hours': "[['Sunday', '11:

In [18]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
llm = OpenAI(openai_api_key=api_key)

vectordb = Chroma(persist_directory='./chroma_db_res', embedding_function=embeddings)

retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [24]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

In [25]:
qa = RetrievalQA.from_chain_type(llm,
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=True)

In [20]:
query = "I'm looking for mexican food. What could you suggest to me?"
result = qa({"query": query})
result['result']

' I would recommend any of the Mexican restaurants listed, as they all offer takeout and other service options, as well as a variety of Mexican food options.'

In [21]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

template = """You are a restaurent recommender system that help users to find restaurents that match their preferences. 
Use the following pieces of context to answer the question at the end. 
For each question, suggest three restaurents, with a short description and the reason why the user migth like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""


PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}

llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0) 

qa = RetrievalQA.from_chain_type(llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

query = "I like mexican food, suggest me something similar but not mexican"
result = qa({'query':query})
print(result['result'])

/Users/nachiket/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


1. Sushi Sake - A Japanese restaurant known for its fresh sushi and sashimi options. You might enjoy the variety of flavors and textures in Japanese cuisine.
2. Pizzeria Bella - An Italian restaurant famous for its wood-fired pizzas and homemade pasta dishes. You might appreciate the hearty and comforting Italian flavors.
3. Curry House - An Indian restaurant offering a wide range of flavorful curries and traditional Indian dishes. You might enjoy the bold spices and aromatic flavors of Indian cuisine.


In [25]:
result['source_documents'][0].metadata['gmap_id']

'0x8640b7e4cccbecb7:0xfae538ac5948d71c'